# StripeAgentToolkit

本 Notebook 提供了 Stripe Agent Toolkit 的快速入门概述。

您可以在 [Stripe 的发布博客](https://stripe.dev/blog/adding-payments-to-your-agentic-workflows) 或项目的 [PyPi 页面](https://pypi.org/project/stripe-agent-toolkit/) 上了解更多关于 `StripeAgentToolkit` 的信息。

## 概述

### 集成详情

| 类名 | 包名 | 可序列化 | [JS 支持](https://github.com/stripe/agent-toolkit?tab=readme-ov-file#typescript) |  最新包 |
| :--- | :--- | :---: | :---: | :---: |
| StripeAgentToolkit | [stripe-agent-toolkit](https://pypi.org/project/stripe-agent-toolkit) | ❌ | ✅ |  ![PyPI - Version](https://img.shields.io/pypi/v/stripe-agent-toolkit?style=flat-square&label=%20) |


## 设置

这个外部管理包托管在 `stripe-agent-toolkit` 项目中，该项目由 Stripe 团队管理。

您可以配合 langgraph 为以下示例安装它，使用 `pip`：

In [1]:
%pip install --quiet -U langgraph stripe-agent-toolkit


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### 凭证

除了安装该包，您还需要配置集成，将您的 Stripe 账户的密钥（可在您的 [Stripe Dashboard](https://dashboard.stripe.com/apikeys) 中找到）添加进去。

In [ ]:
import getpass
import os

if not os.environ.get("STRIPE_SECRET_KEY"):
    os.environ["STRIPE_SECRET_KEY"] = getpass.getpass("STRIPE API key:\n")

设置 [LangSmith](https://smith.langchain.com/) 以获得一流的可观察性也很有帮助（但非必需）：

In [3]:
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

## 实例化

这里我们演示如何创建 Stripe Toolkit 的实例

In [ ]:
from stripe_agent_toolkit.crewai.toolkit import StripeAgentToolkit

stripe_agent_toolkit = StripeAgentToolkit(
    secret_key=os.getenv("STRIPE_SECRET_KEY"),
    configuration={
        "actions": {
            "payment_links": {
                "create": True,
            },
        }
    },
)

## Agent

Here's how to use the toolkit to create a basic agent in langgraph:

In [ ]:
from langchain_anthropic import ChatAnthropic
from langgraph.prebuilt import create_react_agent

llm = ChatAnthropic(
    model="claude-3-5-sonnet-20240620",
)

langgraph_agent_executor = create_react_agent(llm, stripe_agent_toolkit.get_tools())

input_state = {
    "messages": """
        Create a payment link for a new product called 'test' with a price
        of $100. Come up with a funny description about buy bots,
        maybe a haiku.
    """,
}

output_state = langgraph_agent_executor.invoke(input_state)

print(output_state["messages"][-1].content)